# PyTorch music classification

## Importing packages
### System and utility

In [4]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

### Importing PyTorch


In [5]:
import torch
from torch import optim
from torch import nn as nn

## Global constants
* CHUNK_SIZE defines the number of seconds to take from a song in order to train and classify
* BATCH_SIZE determines how many different pieces of music are seen by the machine learning algorithm at one point in time
* LEARNING_RATE is a parameter for the learning algorithm telling the algorithm "how fast or how much it is supposed to learn from a set of samples"
* ITERATIONS sets the number of overall chunks of music the neural network is going to see and learn from

In [6]:
CHUNK_SIZE = 20000
BATCH_SIZE = 512
LEARNING_RATE = 1e-3
ITERATIONS = 5000

### Check if machine is ready for GPU use
Note that using GPU yields heavy speed-ups as hardware is optimized for most machine learning operations

In [7]:
is_gpu = torch.cuda.is_available()
print("GPU detected: ", is_gpu)

('GPU detected: ', True)


## Data loader
Loading the data from a wav file and converting it into a numerical (float) form that is usable to a neural network. Here, the machine learning is going to be done on "raw" audio data. Alternatives could be "visual representations of soundwaves"

In [8]:
from scipy.io import wavfile

def random_wav_slice(path, slice_size, debug=False, category="bla", downsample=7):
    sample_rate, data = wavfile.read(path)
    data = data[::7]

    slice_start = random.randint(0, data.shape[0] - slice_size - 1)
    slice_end = slice_start + slice_size

    convert_16_bit = float(2**15)
    data = data[slice_start:slice_end]/(convert_16_bit+1.0)

    data = data.astype(np.float32)

    return data

def generate_categories(path):
    categories = dict.fromkeys(os.listdir(path))

    for c in categories:
        c_abspath = os.path.join(path, c)
        audio_files = filter(lambda n: n.endswith(".wav"), os.listdir(c_abspath))
        categories[c] = [os.path.join(c_abspath, au) for au in audio_files]

    return categories

def random_sample(categories, slice_size=20000, debug=False, batch_size=128):
    sample_tensors = []
    target_tensors = []

    for batch_i in xrange(batch_size):

        category = random.choice(list(categories.keys()))
        category_index = sorted(list(categories.keys())).index(category)

        random_file = random.choice(list(categories[category]))
        sample_data = random_wav_slice(random_file, slice_size, category=category)

        target_tensors.append(category_index)
        sample_tensors.append(sample_data)

    sample_tensors = torch.from_numpy(np.array(sample_tensors, dtype=np.float32))
    target_tensors = torch.from_numpy(np.array(target_tensors, dtype=np.int64))

    return target_tensors, sample_tensors

def print_predictions(output_tensor, categories):
    values, indexes = output_tensor.topk(3, 1, True)

    for i in range(3):
        value = values[0][i].data[0]
        index = indexes[0][i].data[0]
        name = sorted(list(categories.keys()))[index]
        print("{}: {} ({})".format(i, name, value))

    print(80*"#")
    print("\n\n")

### TODO: PLAY AUDIO SAMPLES HERE

## Defining the neural network
A very simple neural network with 3 overall hierarchical layers. From one layer to another all neurons are connected to all other neurons, usually referred to as "fully-connected" or "Multi-layer perceptron (MLP)". In the last layer there is as many output neurons as there is classes to distinguish in the task. This is motivated from the concept of "grandmother-cells" in Biology, where it is assumed that if you see your grandmother, one particular neuron is activated. In our example, that means that we have one neuron for "rock", one for "blues" etc. This is one of the simplest neural networks one can imagine as it doesn't have any complicated structure. 

In PyTorch neural networks are easily defined through the high-level AP. You decide on the amount of neurons ("hidden-size") in each layer and the connectivity pattern (linear = everything is connected). You then define a "forward" function in which the ordering of the layers is defined. PyTorch then takes care of optimization and updating of the network

In [3]:
class MLP(nn.Module):

    def __init__(self, chunk_size, num_classes, hid1_size=300, hid2_size=128):
        super(MLP, self).__init__()

        self.lin1 = nn.Linear(chunk_size, hid1_size)
        self.act1 = nn.ReLU(True)

        self.lin2 = nn.Linear(hid1_size, hid2_size)
        self.act2 = nn.ReLU(True)

        self.lin3 = nn.Linear(hid2_size, num_classes)

    def forward(self, x):
        x = self.act1(self.lin1(x))
        x = self.act2(self.lin2(x))
        x = self.lin3(x)
        return x

NameError: name 'nn' is not defined

## Initializing neural network model, data loader and definition of how to train the neural network

### data

In [7]:
categories = generate_categories("../../../datasets/gtzan")
CLASSES = categories.keys() # ["rock", "blues", ...]

### Initializing the neural network model
Constructing the network defined earlier for the first time. This network has not seen any data yet and produces completely random output as all concepts are new at this point in time. 

In [8]:
model = MLP(CHUNK_SIZE, len(CLASSES), 256, 128)
print(model)

MLP (
  (lin1): Linear (20000 -> 256)
  (act1): ReLU (inplace)
  (lin2): Linear (256 -> 128)
  (act2): ReLU (inplace)
  (lin3): Linear (128 -> 10)
)


### Select a loss function (to train the network) and an optimization algorithm
There is many optimization algorithms and loss functions. PyTorch has a catalogue of these and specifying these is only two lines of code. 

In [9]:
optimizer = optim.Adam(model.parameters(), LEARNING_RATE, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

### Tell the neural network model and optimizer to use GPU acceleration if available.

In [10]:
if is_gpu:
    model = model.cuda()
    loss_fn = loss_fn.cuda()

### Set the neural network model into "training" mode, that is tell the neural network to learn from the data it is going to see. Create a list to store intermediate results, so that we can monitor how our neural network is getting better! 

In [11]:
model.train()

losses = []

## Train the neural network model

In [12]:
for i in xrange(ITERATIONS):

    # sample a mini-batch from the data
    target_tensor, sample_tensor = random_sample(categories, slice_size=CHUNK_SIZE, batch_size=BATCH_SIZE)

    # place it on GPU if a GPU is available
    if is_gpu:
        target_tensor = target_tensor.cuda()
        sample_tensor = sample_tensor.cuda()

    # put the data into a PyTorch Autograd Variable that is used for optimization
    input_var = torch.autograd.Variable(sample_tensor)
    target_var = torch.autograd.Variable(target_tensor)

    # do the actual calculation and learning
    optimizer.zero_grad()
    output = model(input_var)
    loss = loss_fn(output, target_var)

    loss.backward()
    optimizer.step()

    correct = 0
    total = 0

    # Check how accuracy progresses 
    
    ## TODO: VISUALIZATION of confusion matrix here?
    if i % 5 == 0:
        print "============== Iteration ", i
        y_test = target_var.cpu().data.numpy()
        y_pred_all = output.cpu().data.numpy()
        y_pred = np.argmax(y_pred_all, axis=1)

        correct += (y_pred == y_test).sum()
        total += y_pred.shape[0]

        cnf_matrix = confusion_matrix(y_test, y_pred)
        print "Loss: ", loss.data[0], ", Acc: ", (correct / float(total))
        print cnf_matrix

============== Iteration  0
Loss:  2.30534148216 , Acc:  0.08984375
[[ 0  0  0  7  0  0  0 37  0  0]
 [ 0  0  0  1  0  0  0 43  0  0]
 [ 0  0  0  6  0  0  0 48  0  0]
 [ 0  0  0  6  0  0  0 44  0  0]
 [ 0  1  0  5  3  0  0 41  1  0]
 [ 0  0  0  1  0  0  0 60  0  0]
 [ 0  0  0  5  1  0  0 50  0  0]
 [ 0  1  0  4  1  0  1 37  0  0]
 [ 0  0  0  4  1  0  0 52  0  0]
 [ 0  0  0  3  0  0  0 48  0  0]]
============== Iteration  5
Loss:  2.28909850121 , Acc:  0.09765625
[[ 0  0  0  0 11  0  0 50  0  0]
 [ 0  0  0  0  0  0  0 48  0  0]
 [ 0  0  0  4  8  0  1 42  0  0]
 [ 0  0  0  2  6  0  2 29  0  0]
 [ 0  0  0  0 16  0  0 33  4  0]
 [ 0  0  0  2  2  0  1 50  0  0]
 [ 0  0  0  2  9  0  3 45  1  0]
 [ 0  0  0  2 12  0  1 29  0  0]
 [ 0  0  0  1  6  0  2 40  0  0]
 [ 0  0  0  1  6  0  0 41  0  0]]
============== Iteration  10
Loss:  2.26434445381 , Acc:  0.1015625
[[ 0  0  0  0  6  0  0 45  0  0]
 [ 0  0  0  0  1  0  0 63  0  0]
 [ 0  0  0  0  3  0  1 41  0  0]
 [ 0  0  0  0  6  0  0 37  0  0]
 [

============== Iteration  105
Loss:  2.04841327667 , Acc:  0.236328125
[[ 2  8  0  1  5 17  6  1  7  4]
 [ 0 39  0  0  0  4  1  0  0  0]
 [ 2  4  2  1 10  9  5  1  7  6]
 [ 4  0  4  0 22 10 11  4  8  7]
 [ 0  3  0  1 43  2  1  3  1  4]
 [ 0 15  1  0  3 17  2  0  1 10]
 [ 0  3  0  0 14 15  4  0  5  3]
 [ 1  0  2  0 29  5  2  3  1  3]
 [ 1  1  1  1 21  2  6  3  8  7]
 [ 2  3  0  0 10 20  5  1  8  3]]
============== Iteration  110
Loss:  2.05128645897 , Acc:  0.25
[[ 0  4  1  7  2 13  2  5  3  8]
 [ 0 49  1  0  0  3  0  0  0  3]
 [ 0  6  7  4  4 14  1  8  3 12]
 [ 0  2  1  6  8  5  0 14  4  7]
 [ 2  0  0  7 10  2  4 15  5  1]
 [ 1 14  1  2  1 19  0  3  2  9]
 [ 0  4  0  3  5 18  3  8  4  1]
 [ 1  1  1  2  7  1  1 15  4  3]
 [ 1  3  4 13 11  9  4 11  9  5]
 [ 1  6  2  5  4 17  2  5  3 10]]
============== Iteration  115
Loss:  1.96636414528 , Acc:  0.26953125
[[ 2  2  6  7  1 15  5  9  1  0]
 [ 0 46  1  0  0  6  0  0  0  0]
 [ 0  5  6 14  1 13  1  8  2  0]
 [ 7  0  5  9  1  5  4 14  2  0]
 

============== Iteration  210
Loss:  1.92919135094 , Acc:  0.265625
[[15  6 18  0  2 17  3  1  4  1]
 [ 3 38  0  0  0  5  0  0  0  0]
 [11  3 15  1  6  5  0  3  2  5]
 [11  5 14  3 13  3  1  5  1  0]
 [ 7  0  1  2 27  0  0  6  4  1]
 [ 4 13  7  1  1 13  1  0  0  0]
 [14  2 10  4  7 12  4  7  5  0]
 [ 3  0  8  0 15  2  1 13  3  0]
 [13  1  4  1  6  8  3  3  7  0]
 [ 5  5 16  4  2 10  1  2  2  1]]
============== Iteration  215
Loss:  1.87746787071 , Acc:  0.314453125
[[ 2  3  6  4  0 11  1 15  4  1]
 [ 0 50  1  0  0  4  0  0  2  0]
 [ 6  5 12  8  1  8  0 14  5  0]
 [ 7  0 11 11  3  6  0  8  9  2]
 [ 2  0  2  2 17  0  2 15  4  0]
 [ 3 15  4  0  0 24  0  1  1  2]
 [ 5  1  4  2  3 11  3 11  7  0]
 [ 3  1  3  1  5  0  0 30  2  0]
 [ 2  4  6  8  3  3  1  7  9  3]
 [ 3  4 11  3  1 17  2 11  5  3]]
============== Iteration  220
Loss:  1.8610342741 , Acc:  0.296875
[[ 3  2  6  1  2 11  4  6  5  4]
 [ 0 48  1  0  0  6  0  0  0  2]
 [ 3  3 22  1  1  6  0  7  2  5]
 [ 4  1  7  3  1  8  2  8  7  2]


============== Iteration  315
Loss:  1.82586026192 , Acc:  0.3125
[[17  1  2  4  5 15  5  4  0  1]
 [ 2 43  0  0  0  5  1  0  1  0]
 [10  1  1  1  4  6  6  0  2  3]
 [14  0  3  6  5  6 10  5  6  3]
 [ 1  0  0  1 35  0  4  4  3  0]
 [ 6 14  1  2  0 22  3  1  1  2]
 [ 4  1  2  1 13  4 11  4  6  5]
 [ 4  2  2  0 26  3  3 15  2  1]
 [ 9  1  2  4  8  2 14  0  8  3]
 [17  4  2  1  2 14  5  4  3  2]]
============== Iteration  320
Loss:  1.90803027153 , Acc:  0.30859375
[[ 8  3 11  6  4 12  6  0  3  2]
 [ 0 43  2  1  0  6  1  0  1  0]
 [ 8  4 15  2  2  6  2  3  2  1]
 [ 8  1  8  7  2 11  2  3  7  2]
 [ 2  0  2  2 23  2  3  6  4  0]
 [ 5 10  1  0  0 22  1  0  5  1]
 [ 5  0  2  6  7 10  6  7  9  4]
 [ 5  1  4  5 23  2  2 15  4  2]
 [ 3  2 10  3  5  5  4  6 14  2]
 [10  4  6  5  2  7  2  2  2  5]]
============== Iteration  325
Loss:  1.83211553097 , Acc:  0.291015625
[[ 4  2 27  3  1 11  2  6  9  8]
 [ 1 36  0  0  0  2  0  0  1  0]
 [ 1  6 16  2  1  4  0  8  5  6]
 [ 1  2 15  2  1  1  0  9  1  1]

============== Iteration  420
Loss:  1.7370351553 , Acc:  0.3515625
[[ 3  1 12  2  2  9  3 10  2  4]
 [ 1 58  1  1  0  8  0  0  0  1]
 [ 5  3 22  8  2  8  0  2  1  3]
 [ 4  0 13 10  2  4  2  8  9  5]
 [ 0  0  1  4 28  0  1  9  3  1]
 [ 2  8  6  0  0 28  2  0  2  3]
 [ 1  0  5  2  7 11  6 12  9  1]
 [ 1  2  4  7 12  0  4 15  1  1]
 [ 0  1  7  4  4  1  6  5  9  1]
 [ 2  3  6  4  3 14  2  4  7  1]]
============== Iteration  425
Loss:  1.79805922508 , Acc:  0.341796875
[[ 8  0 10  2  0 10  6  4  2  8]
 [ 1 37  0  0  0  8  0  0  1  1]
 [ 3  2 22  2  2 14  2  1  5  5]
 [ 1  0 11 11  0  4  4  3  7  3]
 [ 0  0  0  1 27  1  1 10 10  0]
 [ 0 10  9  1  0 24  0  2  3  1]
 [ 3  0  7  5  7 12  6  2 11  1]
 [ 0  1  3  7 11  2  5 16  6  1]
 [ 4  2  6  2  6  5  4  2 17  1]
 [ 6  2  7  3  2 13  4  4  9  7]]
============== Iteration  430
Loss:  1.79742324352 , Acc:  0.34375
[[11  2  3  6  2 14  3  2  2  5]
 [ 0 50  0  1  0  9  1  0  1  0]
 [ 6  3 10  4  5 13  0  3  4  2]
 [ 8  0  8 12  3  4  2  9  7  1]


...

============== Iteration  4515
Loss:  1.18850409985 , Acc:  0.568359375
[[25  0  8  5  2  7  3  1  0  2]
 [ 0 38  0  0  0  6  0  0  0  1]
 [ 1  2 26  4  0  7  1  3  1  4]
 [ 2  1  8 21  1  1  2  2  8  9]
 [ 0  2  4  2 34  0  0  2  3  0]
 [ 0  4  3  1  0 31  1  0  2  0]
 [ 2  0  2  6  5  1 18 10  2  6]
 [ 2  0  4  6  4  0  3 39  2  2]
 [ 0  1  2  7  2  0  1  4 37  2]
 [ 1  0 10  6  0  6  3  0  3 22]]
============== Iteration  4520
Loss:  1.12336528301 , Acc:  0.59375
[[25  0  9  4  0  4  5  4  0  0]
 [ 0 44  0  0  0 10  0  0  1  0]
 [ 3  3 27  7  1  3  3  2  2  5]
 [ 4  0  1 21  4  2  6  1  5  3]
 [ 2  1  1  2 39  2  3  8  3  0]
 [ 2  6  2  2  0 42  1  0  1  0]
 [ 3  1  4  6  3  3 31  3  4  8]
 [ 1  0  1  4  2  0  3 23  1  1]
 [ 1  0  3  5  0  1  0  0 34  0]
 [ 3  0  9  2  1  0  5  0  2 18]]
============== Iteration  4525
Loss:  1.11958491802 , Acc:  0.583984375
[[21  0  2  3  2  5  4  8  4  2]
 [ 0 39  0  0  0  4  0  0  0  2]
 [ 2  1 31  2  2  7  1  0  4  7]
 [ 2  1  2 22  6  0  4  4  

============== Iteration  4620
Loss:  1.19381797314 , Acc:  0.56640625
[[25  2  5  2  2  5  3  2  7  2]
 [ 0 47  0  0  0  4  0  0  0  0]
 [ 3  0 26  4  2  3  4  2  1  3]
 [ 6  1  4 23  4  3  3  0  7  6]
 [ 0  1  1  4 31  0  2  4  2  1]
 [ 2  7  2  1  0 28  1  0  0  4]
 [ 2  1  4  6  6  3 20  2  3  8]
 [ 1  1  3  4  4  1  5 28  4  2]
 [ 1  1  1  3  2  1  2  1 28  1]
 [ 4  0 11  2  1  2  1  0  6 34]]
============== Iteration  4625
Loss:  1.16968715191 , Acc:  0.587890625
[[23  0  3  0  1  2  5  2  0  4]
 [ 1 51  0  0  0  8  1  0  0  1]
 [ 5  0 24  6  1  7  1  1  4  7]
 [ 3  1  3 23  2  0  2  2  6  2]
 [ 0  1  2  2 29  0  3  5  3  0]
 [ 4  5  5  1  0 28  1  2  1  1]
 [ 2  0  2  8  1  6 24  5  6  5]
 [ 2  0  6  3  1  0  3 34  0  0]
 [ 1  1  4  9  2  1  1  3 42  4]
 [ 2  0  3  2  2  4  1  3  1 23]]
============== Iteration  4630
Loss:  1.20714640617 , Acc:  0.568359375
[[27  0  6  7  3  5  4  3  1  0]
 [ 0 40  0  0  0  3  0  0  0  2]
 [ 2  0 25  6  1  8  1  1  1  5]
 [ 2  0  6 26  2  0  5  

============== Iteration  4725
Loss:  1.23667895794 , Acc:  0.544921875
[[27  1  3  7  0  3  5  4  4  3]
 [ 0 44  0  0  0  1  0  0  0  2]
 [ 2  3 18  5  2  5  1  1  1  6]
 [ 4  1  2 21  4  3  1  1  4  6]
 [ 0  1  0  3 43  0  2  5  3  0]
 [ 1  5  1  0  1 33  1  0  1  6]
 [ 1  0  2  1  2  4 13  4  6 12]
 [ 2  0  3  5  6  0  4 33  1  5]
 [ 2  0  1  9  2  2  3  1 30  3]
 [ 3  2 10  7  0  2  6  4  3 17]]
============== Iteration  4730
Loss:  1.2153660059 , Acc:  0.576171875
[[36  0  0  5  2  2  4  3  1  3]
 [ 1 37  0  0  0  6  0  0  0  2]
 [ 6  1 28  7  2  4  2  2  2  2]
 [ 5  0  1 24  1  3  8  2  8  5]
 [ 0  0  0  5 33  1  2  2  5  1]
 [ 2  4  0  0  0 36  1  1  2  1]
 [ 3  0  1  6  2  2 13  5  4  7]
 [ 0  1  2  4  5  0  2 31  0  5]
 [ 1  1  1  6  5  1  1  0 34  1]
 [ 4  1 11  6  0  0  6  2  4 23]]
============== Iteration  4735
Loss:  1.12924706936 , Acc:  0.60546875
[[27  0  7  6  1  0  3  0  1  2]
 [ 1 43  0  0  0 10  0  0  0  1]
 [ 3  1 28  2  0  4  0  0  0  3]
 [ 3  1  6 22  2  0  3  4

============== Iteration  4830
Loss:  1.18252170086 , Acc:  0.5859375
[[28  0  2  0  2  3  4  2  4  0]
 [ 2 45  0  0  0  6  0  0  1  1]
 [ 2  0 20  4  1  3  0  2  1  5]
 [ 3  0  4 23  3  0  6  2  9  2]
 [ 0  1  4  3 33  1  2  4  4  1]
 [ 2  3  1  2  0 42  2  0  2  0]
 [ 4  0  2  7  5  4 21  8  4  4]
 [ 3  1  3  1  5  0  2 40  3  1]
 [ 2  1  1  4  3  2  1  3 26  3]
 [ 3  1  7  7  2  3  3  0  3 22]]
============== Iteration  4835
Loss:  1.07694983482 , Acc:  0.640625
[[25  1  2  4  1  5  3  3  1  1]
 [ 0 56  1  0  0  5  0  0  0  0]
 [ 2  0 29  7  0  3  0  1  3  6]
 [ 2  0  4 26  1  1  5  3  7  4]
 [ 0  1  2  6 29  1  3  3  2  0]
 [ 3  5  0  1  0 35  1  0  0  1]
 [ 2  0  1  4  0  2 27  4  4  7]
 [ 0  0  7  2  2  0  3 33  2  1]
 [ 1  0  1  5  0  1  3  3 31  2]
 [ 1  0  4  8  0  5  4  0  0 37]]
============== Iteration  4840
Loss:  1.11870753765 , Acc:  0.591796875
[[24  0  4  3  0  4  6  5  2  1]
 [ 0 46  1  0  0  2  0  0  1  0]
 [ 4  0 30  3  0  6  1  3  2 11]
 [ 1  1  3 18  1  1  7  4  7

============== Iteration  4935
Loss:  1.16715276241 , Acc:  0.5625
[[36  1  3  8  1  2  2  5  0  0]
 [ 1 49  3  0  0  9  0  0  0  1]
 [ 5  2 32  5  1  4  1  3  0  6]
 [ 2  0  4 22  3  1  5  4  8  2]
 [ 0  0  0  2 26  1  2  4  2  2]
 [ 1  1  2  0  1 27  0  0  0  3]
 [ 3  0  4  6  6  1 20  6  1 10]
 [ 1  0  2  4  3  1  8 28  2  1]
 [ 1  0  2  8  3  3  3  2 30  4]
 [ 1  2  5  3  1  5  6  1  2 18]]
============== Iteration  4940
Loss:  1.15560173988 , Acc:  0.578125
[[26  1  8  3  0  3  4  7  2  2]
 [ 0 45  1  0  0  2  0  0  0  1]
 [ 2  0 33  1  2  3  4  3  0  5]
 [ 1  1  6 30  4  1  3  5  7  3]
 [ 0  0  0  2 28  1  4  2  4  2]
 [ 2  3  8  1  0 27  1  2  1  1]
 [ 1  0  8  5  3  1 21  4  3  5]
 [ 0  2  6  4  4  0  0 31  2  0]
 [ 0  0  4  2  3  3  0  1 30  1]
 [ 3  0 12  7  0  1  7  4  1 25]]
============== Iteration  4945
Loss:  1.17577064037 , Acc:  0.5859375
[[28  0  2  4  1  4  6  3  0  1]
 [ 0 55  1  0  0  6  0  0  1  0]
 [ 7  1 32  7  2  3  1  3  1  7]
 [ 1  0  4 26  3  1  2  3  4  2]
